In [3]:
from __future__ import division
from __future__ import print_function
from linear_gae.evaluation import get_roc_score, clustering_latent_space
from linear_gae.kcore import compute_kcore, expand_embedding
from linear_gae.model import *
from linear_gae.optimizer import OptimizerAE, OptimizerVAE
from linear_gae.preprocessing import *
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time
import networkx as nx
import pickle as pkl
import sys

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

flags = tf.app.flags
FLAGS = flags.FLAGS

tf.app.flags.DEFINE_string('f', '', 'kernel')   # 添加的，不报错

In [5]:
# Select graph dataset
flags.DEFINE_string('dataset', 'cora', 'Name of the graph dataset')

# Select machine learning task to perform on graph
flags.DEFINE_string('task', 'link_prediction', 'Name of the learning task')

# Model
flags.DEFINE_string('model', 'gcn_ae', 'Name of the model')
''' Available Models:

- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer
          GCN encoder and inner product decoder

- gcn_vae: Graph Variational Autoencoder from Kipf and Welling (2016), with
           Gaussian priors, 2-layer GCN encoders for mu and sigma, and inner
           product decoder

- linear_ae: Linear Graph Autoencoder, as introduced in section 3 of NeurIPS
             workshop paper, with linear encoder, and inner product decoder

- linear_vae: Linear Graph Variational Autoencoder, as introduced in section 3
              of NeurIPS workshop paper, with Gaussian priors, linear encoders
              for mu and sigma, and inner product decoder
 
- deep_gcn_ae: Deeper version of Graph Autoencoder, as introduced in section 4
               of NeurIPS workshop paper, with 3-layer GCN encoder, and inner
               product decoder
 
- deep_gcn_vae: Deeper version of Graph Variational Autoencoder, as introduced
                in section 4 of NeurIPS workshop paper, with Gaussian priors,
                3-layer GCN encoders for mu and sigma, and inner product
                decoder
'''

' Available Models:\n\n- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer\n          GCN encoder and inner product decoder\n\n- gcn_vae: Graph Variational Autoencoder from Kipf and Welling (2016), with\n           Gaussian priors, 2-layer GCN encoders for mu and sigma, and inner\n           product decoder\n\n- linear_ae: Linear Graph Autoencoder, as introduced in section 3 of NeurIPS\n             workshop paper, with linear encoder, and inner product decoder\n\n- linear_vae: Linear Graph Variational Autoencoder, as introduced in section 3\n              of NeurIPS workshop paper, with Gaussian priors, linear encoders\n              for mu and sigma, and inner product decoder\n \n- deep_gcn_ae: Deeper version of Graph Autoencoder, as introduced in section 4\n               of NeurIPS workshop paper, with 3-layer GCN encoder, and inner\n               product decoder\n \n- deep_gcn_vae: Deeper version of Graph Variational Autoencoder, as introduced\n                

In [6]:
# Model parameters
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_integer('epochs', 200, 'Number of epochs in training.')
flags.DEFINE_boolean('features', False, 'Include node features or not in encoder')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate (with Adam)')
flags.DEFINE_integer('hidden', 32, 'Number of units in GCN hidden layer(s).')
flags.DEFINE_integer('dimension', 16, 'Dimension of encoder output, i.e. \
                                       embedding dimension')

# Experimental setup parameters
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 5., 'Proportion of edges in validation set \
                                   (for Link Prediction task)')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      (for Link Prediction task)')
flags.DEFINE_boolean('validation', False, 'Whether to report validation \
                                           results at each epoch (for \
                                           Link Prediction task)')
flags.DEFINE_boolean('verbose', True, 'Whether to print comments details.')

# Parameters related to the "degeneracy framework" from IJCAI 2019 paper,
# aiming at scaling-up Graph AE/VAE by training the model only on the k-core
# (smaller) version of the graph, then expanding embedding to remaining nodes
# via simpler and faster heuristics
flags.DEFINE_boolean('kcore', False, 'Whether to run k-core decomposition \
                                      and use the framework. False = model \
                                      will be trained on the entire graph')
flags.DEFINE_integer('k', 2, 'Which k-core to use. Higher k => smaller graphs\
                              and faster (but maybe less accurate) training')
flags.DEFINE_integer('nb_iterations', 10, 'Number of fix point iterations in \
                                           algorithm 2 of IJCAI paper. See \
                                           kcore.py file for details')

# Lists to collect average results
if FLAGS.task == 'link_prediction':
    mean_roc = []
    mean_ap = []
    mean_acc = []
    mean_f1 = []

if FLAGS.kcore:
    mean_time_kcore = []
    mean_time_train = []
    mean_time_expand = []
    mean_core_size = []
mean_time = []

In [7]:
def parse_index_file(filename):
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

def load_data(dataset):
    # Load the data: x, tx, allx, graph
    names = ['x', 'tx', 'allx', 'graph']
    objects = []
    for i in range(len(names)):
        with open("data/ind.{}.{}".format(dataset, names[i]), 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))
    x, tx, allx, graph = tuple(objects)
    test_idx_reorder = parse_index_file("data/ind.{}.test.index".format(dataset))
    test_idx_range = np.sort(test_idx_reorder)

    features = sp.vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    graph = nx.from_dict_of_lists(graph)
    adj = nx.adjacency_matrix(graph)
    return adj, features

In [8]:
# Load graph dataset
if FLAGS.verbose:
    print("Loading data...")
adj_init, features_init = load_data(FLAGS.dataset)

Loading data...


In [9]:
# The entire training+test process is repeated FLAGS.nb_run times
for i in range(FLAGS.nb_run):

    if FLAGS.task == 'link_prediction' :
        if FLAGS.verbose:
            print("Masking test edges...")
        # Edge Masking for Link Prediction: compute Train/Validation/Test set
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges(adj_init, FLAGS.prop_test, FLAGS.prop_val)

    # Start computation of running times
    t_start = time.time()

    # Degeneracy Framework / K-Core Decomposition
    if FLAGS.kcore:
        if FLAGS.verbose:
            print("Starting k-core decomposition of the graph")
        # Save adjacency matrix of un-decomposed graph
        # (needed to embed nodes that are not in k-core, after GAE training)
        adj_orig = adj
        # Get the (smaller) adjacency matrix of the k-core subgraph,
        # and the corresponding nodes
        adj, nodes_kcore = compute_kcore(adj, FLAGS.k)
        # Get the (smaller) feature matrix of the nb_core graph
        if FLAGS.features:
            features = features_init[nodes_kcore,:]
        # Flag to compute k-core decomposition's running time
        t_core = time.time()
    elif FLAGS.features:
        features = features_init

    # Preprocessing and initialization
    if FLAGS.verbose:
        print("Preprocessing and Initializing...")
    # Compute number of nodes
    num_nodes = adj.shape[0]
    # If features are not used, replace feature matrix by identity matrix
    if not FLAGS.features:
        features = sp.identity(adj.shape[0])
    # Preprocessing on node features
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]

    # Define placeholders
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape = ())
    }

    # Create model
    model = None
    if FLAGS.model == 'gcn_ae':
        # Standard Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # Standard Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes,
                            features_nonzero)
    elif FLAGS.model == 'linear_ae':
        # Linear Graph Autoencoder
        model = LinearModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'linear_vae':
        # Linear Graph Variational Autoencoder
        model = LinearModelVAE(placeholders, num_features, num_nodes,
                               features_nonzero)
    elif FLAGS.model == 'deep_gcn_ae':
        # Deep (3-layer GCN) Graph Autoencoder
        model = DeepGCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'deep_gcn_vae':
        # Deep (3-layer GCN) Graph Variational Autoencoder
        model = DeepGCNModelVAE(placeholders, num_features, num_nodes,
                                features_nonzero)
    else:
        raise ValueError('Undefined model!')

    # Optimizer
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0]
                                                - adj.sum()) * 2)
    with tf.name_scope('optimizer'):
        # Optimizer for Non-Variational Autoencoders
        if FLAGS.model in ('gcn_ae', 'linear_ae', 'deep_gcn_ae'):
            opt = OptimizerAE(preds = model.reconstructions,
                              labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                            validate_indices = False), [-1]),
                              pos_weight = pos_weight,
                              norm = norm)
        # Optimizer for Variational Autoencoders
        elif FLAGS.model in ('gcn_vae', 'linear_vae', 'deep_gcn_vae'):
            opt = OptimizerVAE(preds = model.reconstructions,
                               labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                             validate_indices = False), [-1]),
                               model = model,
                               num_nodes = num_nodes,
                               pos_weight = pos_weight,
                               norm = norm)

    # Normalization and preprocessing on adjacency matrix
    adj_norm = preprocess_graph(adj)
    adj_label = sparse_to_tuple(adj + sp.eye(adj.shape[0]))

    # Initialize TF session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Model training
    if FLAGS.verbose:
        print("Training...")

    for epoch in range(FLAGS.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_label, features,
                                        placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})
        # Weights update
        outs = sess.run([opt.opt_op, opt.cost, opt.accuracy],
                        feed_dict = feed_dict)
        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display epoch information
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
                  "time=", "{:.5f}".format(time.time() - t))
            # Validation, for Link Prediction
            if not FLAGS.kcore and FLAGS.validation and FLAGS.task == 'link_prediction':
                feed_dict.update({placeholders['dropout']: 0})
                emb = sess.run(model.z_mean, feed_dict = feed_dict)
                feed_dict.update({placeholders['dropout']: FLAGS.dropout})
                val_roc, val_ap = get_roc_score(val_edges, val_edges_false, emb)
                print("val_roc=", "{:.5f}".format(val_roc), "val_ap=", "{:.5f}".format(val_ap))

    # Flag to compute Graph AE/VAE training time
    t_model = time.time()

    # Compute embedding

    # Get embedding from model
    emb = sess.run(model.z_mean, feed_dict = feed_dict)

    # If k-core is used, only part of the nodes from the original
    # graph are embedded. The remaining ones are projected in the
    # latent space via the expand_embedding heuristic
    if FLAGS.kcore:
        if FLAGS.verbose:
            print("Propagation to remaining nodes...")
        # Project remaining nodes in latent space
        emb = expand_embedding(adj_orig, emb, nodes_kcore, FLAGS.nb_iterations)
        # Compute mean running times for K-Core, GAE Train and Propagation steps
        mean_time_expand.append(time.time() - t_model)
        mean_time_train.append(t_model - t_core)
        mean_time_kcore.append(t_core - t_start)
        # Compute mean size of K-Core graph
        # Note: size is fixed if task is node clustering, but will vary if
        # task is link prediction due to edge masking
        mean_core_size.append(len(nodes_kcore))

    # Compute mean total running time
    mean_time.append(time.time() - t_start)

    # Test model
    if FLAGS.verbose:
        print("Testing model...")
    # Link Prediction: classification edges/non-edges
    if FLAGS.task == 'link_prediction':
        # Get ROC and AP scores
        ap_score, roc_score, acc_score, f1_score = get_roc_score(test_edges, test_edges_false, emb)
        # Report scores
        mean_ap.append(ap_score)
        mean_roc.append(roc_score)
        mean_acc.append(acc_score)
        mean_f1.append(f1_score)

Masking test edges...
Preprocessing and Initializing...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
targets is deprecated, use labels instead

Training...
Epoch: 0001 train_loss= 0.79734 time= 0.28095
Epoch: 0002 train_loss= 0.79659 time= 0.20303
Epoch: 0003 train_loss= 0.79465 time= 0.20499
Epoch: 0004 train_loss= 0.79072 time= 0.20712
Epoch: 0005 train_loss= 0.78390 time= 0.21110
Epoch: 0006 train_loss= 0.77335 time= 0.19691
Epoch: 0007 train_loss= 0.75884 time= 0.20680
Epoch: 0008 train_loss= 0.74161 time= 0.20139
Epoch: 0009 train_loss= 0.72545 time= 0.19580
Epoch: 0010 train_loss= 0.71731 time= 0.19583
Epoch: 0011 train_loss= 0.72096 time= 0.20346
Epoch: 0012 train_loss= 0.72332 time= 0.20264
Epoch: 0013 train_loss= 0.71684 time= 0.20845
Epoch: 0014 train_loss= 0.70622 time= 0.19800
Epoch: 0015 train_loss= 0.69671 time= 0.19550
Epoch: 0016 train_loss= 0.69036 time= 0.20096
Epoch

Epoch: 0131 train_loss= 0.43149 time= 0.18950
Epoch: 0132 train_loss= 0.43112 time= 0.18450
Epoch: 0133 train_loss= 0.43075 time= 0.19049
Epoch: 0134 train_loss= 0.43039 time= 0.18849
Epoch: 0135 train_loss= 0.43003 time= 0.18650
Epoch: 0136 train_loss= 0.42967 time= 0.19059
Epoch: 0137 train_loss= 0.42932 time= 0.18940
Epoch: 0138 train_loss= 0.42897 time= 0.19348
Epoch: 0139 train_loss= 0.42862 time= 0.20346
Epoch: 0140 train_loss= 0.42827 time= 0.20552
Epoch: 0141 train_loss= 0.42792 time= 0.19149
Epoch: 0142 train_loss= 0.42756 time= 0.19249
Epoch: 0143 train_loss= 0.42721 time= 0.18949
Epoch: 0144 train_loss= 0.42686 time= 0.19655
Epoch: 0145 train_loss= 0.42652 time= 0.19632
Epoch: 0146 train_loss= 0.42617 time= 0.20546
Epoch: 0147 train_loss= 0.42583 time= 0.21641
Epoch: 0148 train_loss= 0.42550 time= 0.21143
Epoch: 0149 train_loss= 0.42517 time= 0.20645
Epoch: 0150 train_loss= 0.42486 time= 0.20445
Epoch: 0151 train_loss= 0.42455 time= 0.21143
Epoch: 0152 train_loss= 0.42425 ti

Epoch: 0108 train_loss= 0.43944 time= 0.19282
Epoch: 0109 train_loss= 0.43908 time= 0.18685
Epoch: 0110 train_loss= 0.43872 time= 0.19646
Epoch: 0111 train_loss= 0.43836 time= 0.18498
Epoch: 0112 train_loss= 0.43801 time= 0.18998
Epoch: 0113 train_loss= 0.43765 time= 0.19477
Epoch: 0114 train_loss= 0.43730 time= 0.20163
Epoch: 0115 train_loss= 0.43694 time= 0.19746
Epoch: 0116 train_loss= 0.43659 time= 0.20224
Epoch: 0117 train_loss= 0.43623 time= 0.19865
Epoch: 0118 train_loss= 0.43587 time= 0.19258
Epoch: 0119 train_loss= 0.43551 time= 0.20001
Epoch: 0120 train_loss= 0.43515 time= 0.20052
Epoch: 0121 train_loss= 0.43477 time= 0.20451
Epoch: 0122 train_loss= 0.43439 time= 0.19662
Epoch: 0123 train_loss= 0.43401 time= 0.19267
Epoch: 0124 train_loss= 0.43361 time= 0.19847
Epoch: 0125 train_loss= 0.43321 time= 0.20047
Epoch: 0126 train_loss= 0.43280 time= 0.19993
Epoch: 0127 train_loss= 0.43239 time= 0.19878
Epoch: 0128 train_loss= 0.43197 time= 0.19079
Epoch: 0129 train_loss= 0.43156 ti

Epoch: 0086 train_loss= 0.44834 time= 0.19178
Epoch: 0087 train_loss= 0.44779 time= 0.18030
Epoch: 0088 train_loss= 0.44726 time= 0.18139
Epoch: 0089 train_loss= 0.44674 time= 0.18359
Epoch: 0090 train_loss= 0.44622 time= 0.19149
Epoch: 0091 train_loss= 0.44571 time= 0.19348
Epoch: 0092 train_loss= 0.44521 time= 0.19847
Epoch: 0093 train_loss= 0.44472 time= 0.19448
Epoch: 0094 train_loss= 0.44423 time= 0.19149
Epoch: 0095 train_loss= 0.44376 time= 0.18833
Epoch: 0096 train_loss= 0.44330 time= 0.18831
Epoch: 0097 train_loss= 0.44284 time= 0.18918
Epoch: 0098 train_loss= 0.44239 time= 0.18375
Epoch: 0099 train_loss= 0.44195 time= 0.19310
Epoch: 0100 train_loss= 0.44152 time= 0.18949
Epoch: 0101 train_loss= 0.44109 time= 0.19094
Epoch: 0102 train_loss= 0.44067 time= 0.19600
Epoch: 0103 train_loss= 0.44026 time= 0.19820
Epoch: 0104 train_loss= 0.43985 time= 0.18693
Epoch: 0105 train_loss= 0.43945 time= 0.19164
Epoch: 0106 train_loss= 0.43906 time= 0.19134
Epoch: 0107 train_loss= 0.43868 ti

Epoch: 0064 train_loss= 0.46226 time= 0.18958
Epoch: 0065 train_loss= 0.46163 time= 0.18865
Epoch: 0066 train_loss= 0.46102 time= 0.18943
Epoch: 0067 train_loss= 0.46041 time= 0.18947
Epoch: 0068 train_loss= 0.45980 time= 0.19089
Epoch: 0069 train_loss= 0.45919 time= 0.18334
Epoch: 0070 train_loss= 0.45858 time= 0.19709
Epoch: 0071 train_loss= 0.45796 time= 0.18409
Epoch: 0072 train_loss= 0.45734 time= 0.18276
Epoch: 0073 train_loss= 0.45673 time= 0.18238
Epoch: 0074 train_loss= 0.45613 time= 0.19160
Epoch: 0075 train_loss= 0.45554 time= 0.18755
Epoch: 0076 train_loss= 0.45497 time= 0.18898
Epoch: 0077 train_loss= 0.45443 time= 0.18785
Epoch: 0078 train_loss= 0.45389 time= 0.19024
Epoch: 0079 train_loss= 0.45337 time= 0.18797
Epoch: 0080 train_loss= 0.45285 time= 0.19852
Epoch: 0081 train_loss= 0.45234 time= 0.18799
Epoch: 0082 train_loss= 0.45182 time= 0.18990
Epoch: 0083 train_loss= 0.45130 time= 0.18241
Epoch: 0084 train_loss= 0.45076 time= 0.18585
Epoch: 0085 train_loss= 0.45022 ti

...

Epoch: 0020 train_loss= 0.64347 time= 0.20136
Epoch: 0021 train_loss= 0.63205 time= 0.19752
Epoch: 0022 train_loss= 0.61909 time= 0.20062
Epoch: 0023 train_loss= 0.60496 time= 0.19454
Epoch: 0024 train_loss= 0.59040 time= 0.19187
Epoch: 0025 train_loss= 0.57631 time= 0.19692
Epoch: 0026 train_loss= 0.56366 time= 0.19306
Epoch: 0027 train_loss= 0.55314 time= 0.18750
Epoch: 0028 train_loss= 0.54503 time= 0.19400
Epoch: 0029 train_loss= 0.53906 time= 0.18895
Epoch: 0030 train_loss= 0.53450 time= 0.19446
Epoch: 0031 train_loss= 0.53058 time= 0.19153
Epoch: 0032 train_loss= 0.52674 time= 0.19481
Epoch: 0033 train_loss= 0.52284 time= 0.19160
Epoch: 0034 train_loss= 0.51895 time= 0.19515
Epoch: 0035 train_loss= 0.51512 time= 0.19866
Epoch: 0036 train_loss= 0.51124 time= 0.19724
Epoch: 0037 train_loss= 0.50718 time= 0.19998
Epoch: 0038 train_loss= 0.50287 time= 0.18894
Epoch: 0039 train_loss= 0.49840 time= 0.19471
Epoch: 0040 train_loss= 0.49401 time= 0.19150
Epoch: 0041 train_loss= 0.48999 ti

Epoch: 0200 train_loss= 0.41025 time= 0.19396
Testing model...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 0.79733 time= 0.25130
Epoch: 0002 train_loss= 0.79660 time= 0.19578
Epoch: 0003 train_loss= 0.79475 time= 0.18655
Epoch: 0004 train_loss= 0.79107 time= 0.18053
Epoch: 0005 train_loss= 0.78474 time= 0.18522
Epoch: 0006 train_loss= 0.77497 time= 0.18057
Epoch: 0007 train_loss= 0.76146 time= 0.18436
Epoch: 0008 train_loss= 0.74504 time= 0.18800
Epoch: 0009 train_loss= 0.72870 time= 0.18425
Epoch: 0010 train_loss= 0.71836 time= 0.18606
Epoch: 0011 train_loss= 0.71938 time= 0.18543
Epoch: 0012 train_loss= 0.72280 time= 0.18874
Epoch: 0013 train_loss= 0.71824 time= 0.18357
Epoch: 0014 train_loss= 0.70808 time= 0.18473
Epoch: 0015 train_loss= 0.69751 time= 0.18705
Epoch: 0016 train_loss= 0.68934 time= 0.19489
Epoch: 0017 train_loss= 0.68358 time= 0.19116
Epoch: 0018 train_loss= 0.67868 time= 0.18078
Epoch: 0019 train_loss= 0.67302 time= 0.1

Epoch: 0177 train_loss= 0.41556 time= 0.18150
Epoch: 0178 train_loss= 0.41544 time= 0.18636
Epoch: 0179 train_loss= 0.41532 time= 0.18346
Epoch: 0180 train_loss= 0.41521 time= 0.18126
Epoch: 0181 train_loss= 0.41510 time= 0.18989
Epoch: 0182 train_loss= 0.41499 time= 0.18426
Epoch: 0183 train_loss= 0.41489 time= 0.18228
Epoch: 0184 train_loss= 0.41479 time= 0.19220
Epoch: 0185 train_loss= 0.41469 time= 0.18595
Epoch: 0186 train_loss= 0.41459 time= 0.18352
Epoch: 0187 train_loss= 0.41449 time= 0.17904
Epoch: 0188 train_loss= 0.41439 time= 0.18793
Epoch: 0189 train_loss= 0.41428 time= 0.18226
Epoch: 0190 train_loss= 0.41418 time= 0.18699
Epoch: 0191 train_loss= 0.41407 time= 0.18467
Epoch: 0192 train_loss= 0.41397 time= 0.19883
Epoch: 0193 train_loss= 0.41386 time= 0.18820
Epoch: 0194 train_loss= 0.41375 time= 0.18868
Epoch: 0195 train_loss= 0.41364 time= 0.18652
Epoch: 0196 train_loss= 0.41352 time= 0.18922
Epoch: 0197 train_loss= 0.41341 time= 0.18670
Epoch: 0198 train_loss= 0.41329 ti

Epoch: 0154 train_loss= 0.41865 time= 0.20402
Epoch: 0155 train_loss= 0.41837 time= 0.19659
Epoch: 0156 train_loss= 0.41809 time= 0.19181
Epoch: 0157 train_loss= 0.41781 time= 0.20140
Epoch: 0158 train_loss= 0.41754 time= 0.19507
Epoch: 0159 train_loss= 0.41726 time= 0.19595
Epoch: 0160 train_loss= 0.41699 time= 0.19475
Epoch: 0161 train_loss= 0.41671 time= 0.19579
Epoch: 0162 train_loss= 0.41643 time= 0.19266
Epoch: 0163 train_loss= 0.41616 time= 0.18913
Epoch: 0164 train_loss= 0.41588 time= 0.19538
Epoch: 0165 train_loss= 0.41561 time= 0.21558
Epoch: 0166 train_loss= 0.41534 time= 0.19880
Epoch: 0167 train_loss= 0.41507 time= 0.20108
Epoch: 0168 train_loss= 0.41480 time= 0.19335
Epoch: 0169 train_loss= 0.41454 time= 0.20393
Epoch: 0170 train_loss= 0.41428 time= 0.19873
Epoch: 0171 train_loss= 0.41403 time= 0.19047
Epoch: 0172 train_loss= 0.41378 time= 0.19851
Epoch: 0173 train_loss= 0.41354 time= 0.19477
Epoch: 0174 train_loss= 0.41331 time= 0.19367
Epoch: 0175 train_loss= 0.41308 ti

Epoch: 0107 train_loss= 0.43442 time= 0.18872
Epoch: 0108 train_loss= 0.43390 time= 0.19248
Epoch: 0109 train_loss= 0.43339 time= 0.18754
Epoch: 0110 train_loss= 0.43289 time= 0.19986
Epoch: 0111 train_loss= 0.43241 time= 0.19506
Epoch: 0112 train_loss= 0.43194 time= 0.19653
Epoch: 0113 train_loss= 0.43148 time= 0.19323
Epoch: 0114 train_loss= 0.43104 time= 0.19618
Epoch: 0115 train_loss= 0.43060 time= 0.19486
Epoch: 0116 train_loss= 0.43017 time= 0.18028
Epoch: 0117 train_loss= 0.42975 time= 0.18449
Epoch: 0118 train_loss= 0.42933 time= 0.18759
Epoch: 0119 train_loss= 0.42892 time= 0.18558
Epoch: 0120 train_loss= 0.42852 time= 0.18910
Epoch: 0121 train_loss= 0.42813 time= 0.18880
Epoch: 0122 train_loss= 0.42775 time= 0.19178
Epoch: 0123 train_loss= 0.42737 time= 0.18945
Epoch: 0124 train_loss= 0.42701 time= 0.18985
Epoch: 0125 train_loss= 0.42667 time= 0.19112
Epoch: 0126 train_loss= 0.42633 time= 0.18444
Epoch: 0127 train_loss= 0.42600 time= 0.18454
Epoch: 0128 train_loss= 0.42569 ti

In [10]:
###### Report Final Results ######

# Report final results
print("\nTest results for", FLAGS.model,
      "model on", FLAGS.dataset, "on", FLAGS.task, "\n",
      "___________________________________________________\n")

if FLAGS.task == 'link_prediction':
    print("AP scores\n", mean_ap)
    print("Mean AP score: ", np.mean(mean_ap),
          "\nStd of AP scores: ", np.std(mean_ap), "\n \n")
    
    print("AUC scores\n", mean_roc)
    print("Mean AUC score: ", np.mean(mean_roc),
          "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")
    
    print("ACC scores\n", mean_acc)
    print("Mean ACC score: ", np.mean(mean_acc),
          "\nStd of ACC scores: ", np.std(mean_acc), "\n \n")
    
    print("F1 scores\n", mean_f1)
    print("Mean F1 score: ", np.mean(mean_f1),
          "\nStd of F1 scores: ", np.std(mean_f1), "\n \n")

if FLAGS.kcore:
    print("Details on degeneracy framework, with k =", FLAGS.k, ": \n \n")

    print("Running times for k-core decomposition\n", mean_time_kcore)
    print("Mean: ", np.mean(mean_time_kcore),
          "\nStd: ", np.std(mean_time_kcore), "\n \n")

    print("Running times for autoencoder training\n", mean_time_train)
    print("Mean: ", np.mean(mean_time_train),
          "\nStd: ", np.std(mean_time_train), "\n \n")

    print("Running times for propagation\n", mean_time_expand)
    print("Mean: ", np.mean(mean_time_expand),
          "\nStd: ", np.std(mean_time_expand), "\n \n")

    print("Sizes of k-core subgraph\n", mean_core_size)
    print("Mean: ", np.mean(mean_core_size),
          "\nStd: ", np.std(mean_core_size), "\n \n")


Test results for gcn_ae model on cora on link_prediction 
 ___________________________________________________

AP scores
 [0.8986684401177076, 0.8769288132694488, 0.8700742895945683, 0.8826454669888419, 0.8718322324755028, 0.8925877426924322, 0.8993076930518, 0.8704754522517385, 0.8719461553621801, 0.8904244308913802]
Mean AP score:  0.88248907166956 
Std of AP scores:  0.011241473103280043 
 

AUC scores
 [0.8626502813894119, 0.8306226573386286, 0.826973416531943, 0.8472251727403332, 0.8296630888384, 0.8627150927702905, 0.8621587950844167, 0.8260444534060182, 0.8261542726902844, 0.8550925542525267]
Mean AUC score:  0.8429299785042254 
Std of AUC scores:  0.01570064670926057 
 

ACC scores
 [0.7409867172675522, 0.6755218216318786, 0.6755218216318786, 0.6935483870967742, 0.7049335863377609, 0.722011385199241, 0.715370018975332, 0.6859582542694497, 0.6650853889943074, 0.698292220113852]
Mean ACC score:  0.6977229601518026 
Std of ACC scores:  0.022432662931509147 
 

F1 scores
 [0.7676